# Deities

- [Devotee Beneftis](#Devotee-Benefits)
- [Analytics](#Analytics)

In [106]:
import os
import glob
import pandas as pd

directory = '/home/ulysses/Documents/Naerk/naerk-wiki/quartz/content/Gods/.data'

os.chdir(directory)
extension = 'tsv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

dataframes = []  # List to store the DataFrames

for csv_file in all_filenames:
    df = pd.read_csv(csv_file, sep='\t')
    dataframes.append(df)

merged_df = pd.merge(dataframes[0], dataframes[1], on='Name', how='left', sort=False)

for i in range(2, len(dataframes)):
    merged_df = pd.merge(merged_df, dataframes[i], on='Name', how='left', sort=False)

# older implementation:
# df = merged_df.set_index('Name').transpose().sort_index(ascending=True)
    
df = merged_df.T.reset_index()
df.columns = df.loc[0]
df = df.drop(0).sort_values(by="Name").reset_index(drop=True)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)


## Deity Characteristics

In [107]:
columns = ['Name', 'Title', 'Category', 'Edicts', 'Portfolio', 'Anathema', 'Followers', 'Pantheon', 'Plane']
df.set_index('Name').rename_axis(None).filter(items=columns)

,Title,Category,Edicts,Portfolio,Anathema,Followers,Pantheon,Plane
Amehan,Destroyer,-,"Destroy material plane, Loss","Destruction, Razing Structures, Pillaging, Civilization's End, Despair",Creating beautiful things,"Warlords, pillagers, invaders, nihilists, savages, goblinoids, titans, giants, Vulga, Erevi",-,Destruction
Anhalt,Unrelenting,Ascended,"Unstoppable Will, Sacrifice, Duty to the end","Sacrifice, Honor, Valor, Martyrdom","Ignore duty or orders, Show fear","Warriors, bodyguards, knights, defenders, dwarves, Hurtsvigsen",Dwarven,"Kelzadam, Fortress of the Just"
Auromia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B'tolen,Raven's Flight,Aspects,"Meet new civilizations, constant learning, wondering and wandering","Unbound travel, Learning, Luck, Exploration",Permanent settlers,"Travelers, wanderers, pilgrims, sailors, elves, tribes","Tribal, Elven, Druidic",Evergrowing Forest
Berohath,Fox's Cunning,Aspects,"Lie and Deceive fools, Take advantage of situations, Spread doubt","Stealth, Thievery, Malice, Luck, Night","Announce intentions, Give back stolen things","Travelers, thieves, bagabonds, beggars, downtrodden, scavengers, rogues, elves, tribes","Tribal, Elven, Goblin",Evergrowing Forest
Crel'vach,The Bear's Fury,Aspects,"Fury of the Wild, Berserker Rage, Unrelenting Combat","Berserkers, warriors, endurance, savagery","Not answering an insult, cowardice","Berserkers, warriors, vanguards, gladiators, orcs, Hurtsvigsen","tribal, orc",Evergrowing Forest
Disiklek,The Plague Lord,Chosen,"Spread Disease, Plague and Poisons, Spoil communities and food, Wait to take its course","Disease, Plague, Poison, Cannibalism, Virulent Fungi and Vermin","Stop a plague, cure a disease or poison","Poisoners, Alchemists, Lunatics, Serpentfolk Crimson Steel","Serpentfolk, Demonic",Abyss
Drokasner,Thunderstrike,Aspects,"Bad Weather, Fury of the Sky, Defiance","Thunder, Storm, Bad Temper","Try to calm or avoid weather, Disrespect the elements","Travellers, mountaineers, sailors, orcs, giants, Hoen","Tribal, Orc, Giant","Elemental Ring, Evergrowing Forest"
Factar,The Compassionate,"Luminous Walkers, Human","Help those in need, Humility, Aid by example","Healing, Farming, Fertility, Wanderers, Poor","Αrrogance, Egotism above others, Cold shoulder","Hospitalers, Healers, Pilgrims, Travelers, Midwives, Herbalists, Priests, Commonfolk, Luminous Order","Luminous Walkers, halfling",Golden Palace
Fallfaunt,Snow's Softness,Aspects,"Cruelty of Cold, Lovestroke, Endurance","Lust, Perdition, Sorrow, Preservance","Change way of life, find new love","Cold sailors, mountaineers, giants, frost creatures, elves, Hoss Glacier","Elemental, tribal, Elven, Giant","Elemental Ring, Evergrowing Forest"


### Devotee Benefits

In [108]:
columns = ['Name', 'Divine Ability', 'Divine Font', 'Divine Skill', 'Favored Weapon', 'Domains', 'Cleric Spells']
df.set_index('Name').rename_axis(None).filter(items=columns)


,Divine Ability,Divine Font,Divine Skill,Favored Weapon,Domains,Cleric Spells
Amehan,Strength or Dexterity,Harm,Athletics,Mace,"Destruction, Decay, Dust, Pain, Sorrow, Void","1st: Phantom Pain, 6th: Disintegrate, 9th: Disjunction"
Anhalt,Constitution or Wisdom,Heal or Harm,Athletics,Warhammer,"Confidence, Duty, Fire, Might, Protection, Vigil, Zeal","1st: True Strike, 4th: Fire Shield, 5th: Mantle of the Magma Heart"
Auromia,NaN,NaN,NaN,NaN,NaN,NaN
B'tolen,Dexterity or Intelligence,Heal or Harm,Society,Longbow,"Air, Change, Freedom, Knowledge, Swarm, Travel","1st: Animal Allies, 2nd: Animal Form, 9th: Shapechange"
Berohath,Dexterity or Intelligence,Harm,Stealth,Dagger,"Darkness, Luck, Moon, Secrecy, Trickery","1st: Invisible Item, 2nd: Invisibility, 5th: Chameleon Coat"
Crel'vach,Strength or Constitution,Heal or Harm,Intimidate,Greataxe,"Confidence, Might, Nature, Protection, Zeal","1st: Magic Fang, 2nd: Enlarge, 4th: Weapon Storm"
Disiklek,Intelligence or Constitution,Harm,Crafting,Scythe,"Decay, Delirium, Pain, Plague, Sorrow","1st: Noxious Vapors, 3rd: Gasping Marsh, 5th: Cloudkill"
Drokasner,Constitution or Wisdom,Harm,Survival,Spear,"Air, Confidence, Lightning, Might","1st: Horizon Thunder Sphere, 5th: Stormburst, 6th: Chain Lightning"
Factar,Wisdom or Charisma,Heal,Medicine,Sickle,"Cities, Family, Healing, Protection, Sun, Travel, Wood","1st: Soothe, 3rd: Cozy Cabin, 6th: Collective Transposition"
Fallfaunt,Dexterity or Charisma,Harm,Performance,Morningstar,"Cold, Confidence, Nightmares, Passion, Sorrow, Water","1st: Chilling Spray, 5th: Cone of Cold, 8th: Polar Ray"


## Analytics

In [114]:
from IPython.display import display, Markdown

excel_out = False # set to true to produce analytic spreadsheet file

if excel_out == True:
    excelname = 'Gods.xlsx'
    df.to_excel(excelname, sheet_name="Data", index=False)
    # df.to_csv("_dataframe.csv")

columns_to_analyse = ['Category', 'Portfolio', 'Followers', 'Pantheon', 'Plane', 'Divine Skill', 'Favored Weapon', 'Domains']

# df.groupby(['Category','Name']).count()

for col in columns_to_analyse:
    explode = df[col].str.split(', ').explode()
    valuecounts = explode.value_counts()
    dfx = valuecounts.rename_axis(col).to_frame('count').reset_index()
    dfx['occurences'] = dfx[col].apply(lambda x: df[df[col].str.contains(x, na=False)]['Name'].tolist())
    dfx['occurences'] = dfx['occurences'].str.join(', ')
    display(Markdown(f"### {col}"))
    display(dfx.set_index(col).rename_axis(None))
    # valuecounts.plot(kind='pie')
    if excel_out == True:
        with pd.ExcelWriter(excelname, engine='openpyxl', mode='a') as writer:
            dfx.to_excel(writer, sheet_name=col, index=False)

ImportError: Pandas requires version '3.1.2' or newer of 'jinja2' (version '3.0.3' currently installed).